In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [220]:
import pandas as pd
df = pd.read_pickle("/content/drive/MyDrive/MCI/filtered_data.pkl")
df.drop_duplicates(subset=['description','title','raw_text'],keep=False, inplace=True)
df.head()

,date,title,tags,description,raw_text
0,2023-02-25,بازگشایی ۷۰ درصد از مجموعههای ورزشی تعطیل شده ...,"[ورزش,سازمان ورزش شهرداری تهران,مجموعه ورزشی,ش...",مدیرعامل سازمان ورزش شهرداری تهران در مورد وضع...,حسین اوجاقی در گفت‌وگو با ایسنا،‌ در مورد آخری...
1,2023-02-25,خبرگزاری فارس - چالش جالب رونالدو با ستاره ازب...,"[رونالدو, چالش رونالدو, در عربستان, چالش جالب ...",دو بازیکن النصر عربستان ابرستاره پرتغالی را به...,به گزارش خبرگزاری فارس، تیم فوتبال النصر امروز...
2,2023-02-25,خبرگزاری فارس - تشییع و تدفین پیکر «اسماعیل اح...,"[اسماعیل احمدی, مشاور وزیر ورزش, تدفین, امامزا...",مراسم تشییع و تدفین اسماعیل احمدی مشاور وزیر و...,به گزارش خبرگزاری فارس از تهران، مراسم تشییع و...
3,2023-02-25,لحظه شلیک موشک کروز دوربرد پاوه,"[موشک کروز,آزمایش موشکی,نیروی هوافضای سپاه,هوا...",فیلمی از لحظه شلیک موشک کروز دوربرد پاوه توسط ...,https://mehrnews.com/xZCFn ۶ اسفند ۱۴۰۱،‏ ۹:۵۷...
7,2023-02-25,معماری ایرانی در خانهای تاریخی + فیلم,"[بنای تاریخی،خانه قدیمی،موزه لوور, بنای تاریخی...",معماری بناهای ایران در جهان شهرت دارد.,شاید بتوان گفت که یکی از زیباترین خانه های قاج...


In [ ]:
# [i for i in df['all_tags'] if len(i) == 0 ]

In [ ]:
!pip install hazm
!pip install clean-text[gpl]

In [221]:
import re
from hazm import *
from cleantext import clean

normalizer = Normalizer(persian_numbers=False, persian_style=False, token_based=True)

def _normalize_text(x):
    """normalize a sentence"""

    # Remove mentions
    x = re.sub('@[^ ]* ', '', x)

    x = re.sub(r"\s+", " ", x)  # remove more than one white spaces with space
    # Remove HTML markup
    x = re.sub('<[^>]*>', '', x)

    x = re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', ' ', x)
    # x = re.sub(r"(\/|\)+", " ", x) 
    
    # regular cleaning
    x = clean(x,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=True,
        no_digits=True,
        no_currency_symbols=True,
        no_punct=True,
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
        replace_with_punct=""
    )

    # normalizing
    x = normalizer.normalize(x)
    return x

df['description'] = df['description'].apply(_normalize_text)

In [25]:
# df['all_tags'] = df.apply (lambda row: row.tags if len(row.tags)!=0 else row.description.split(' '), axis=1)

In [222]:
tagger = POSTagger(model='/content/drive/MyDrive/MCI/postagger.model')
df['POS_tag_description'] = df['description'].apply(lambda x : tagger.tag(word_tokenize(x)))

In [29]:
df['POS_tag_description'][0]

[('مدیرعامل', 'Ne'),
 ('سازمان', 'Ne'),
 ('ورزش', 'Ne'),
 ('شهرداری', 'Ne'),
 ('تهران', 'N'),
 ('در', 'P'),
 ('مورد', 'Ne'),
 ('وضعیت', 'Ne'),
 ('مجموعههای', 'Ne'),
 ('ورزشی', 'AJe'),
 ('تعطیل\u200cشده', 'AJe'),
 ('شهرداری', 'Ne'),
 ('تهران', 'N'),
 ('توضیحاتی', 'N'),
 ('ارائه', 'N'),
 ('کرد', 'V'),
 ('و', 'CONJ'),
 ('گفت', 'V'),
 ('تاکنون', 'ADV'),
 ('درصد', 'N'),
 ('از', 'P'),
 ('مجموعههای', 'Ne'),
 ('ورزشی', 'AJ'),
 ('که', 'CONJ'),
 ('بسته_بودند', 'V'),
 ('مجددا', 'ADV'),
 ('بازگشایی', 'N'),
 ('شدند', 'V'),
 ('و', 'CONJ'),
 ('تنها', 'ADV'),
 ('بخش', 'Ne'),
 ('کوچکی', 'AJ'),
 ('از', 'P'),
 ('مجموعههای', 'Ne'),
 ('ورزشی', 'AJ'),
 ('همچنان', 'ADV'),
 ('بسته', 'AJ'),
 ('هستند', 'V')]

In [223]:
from hazm import stopwords_list

f = open("/content/drive/MyDrive/MCI/STOPWORDS.txt", "r")
STOPWORDS = []
for x in f:
  STOPWORDS.append(x.replace('\n',''))

punctuation_signs = list("?:!.,;")
stopword_list = stopwords_list()

unwanted_POS = ['P','CONJ','V','PRO'] 
stopwords_from_tag = []

def find_stopwords_from_tag(POS_tag):
  for word in POS_tag:
      if word[1] in unwanted_POS:
          stopwords_from_tag.append(word[0])

df['POS_tag_description'].apply(lambda x: find_stopwords_from_tag(x))

filter_words = []
filter_words.extend(punctuation_signs)
filter_words.extend(stopword_list)
stopwords_plus1 = set(filter_words)

filter_words2 = []
filter_words2.extend(STOPWORDS)
filter_words2.extend(stopwords_from_tag)
stopwords_plus2 = set(filter_words2)

stopwords_plus = set.union(stopwords_plus1,stopwords_plus2)

# stopwords_plus = set(filter_words)

df['description_removed_stopwords'] = df['description'].map(lambda x: ' '.join([word for word in x.split() if word not in stopwords_plus]))

In [30]:
# lemmatizer = Lemmatizer()

# adjective_tags = ['JJ','JJR','JJS']

# def lemmetize(POS_tag):
#   lemmatized_text = []
#   for word in POS_tag:
#       if word[1] in adjective_tags:
#           lemmatized_text.append(str(lemmatizer.lemmatize(word[0],pos="a")))
#       else:
#           lemmatized_text.append(str(lemmatizer.lemmatize(word[0]))) #default POS = noun
#   return lemmatized_text

# # df['POS_tag_description'] = df['description'].apply(lambda x : tagger.tag(word_tokenize(x)))
# df['lemmetize_description'] = df['POS_tag_description'].apply(lambda x:lemmetize(x) )

In [65]:
# def remove_stop_words(lemmatized_text):
#   processed_text = []
#   for word in lemmatized_text:
#       if word not in stopwords_plus:
#           processed_text.append(word)
#   return(processed_text) 

# df['lemmetize_description_remove_stopwords'] = df['lemmetize_description'].apply(lambda x:remove_stop_words(x) )

In [224]:
df['description_split'] = df['description_removed_stopwords'].apply(lambda x : x.split(' '))

In [142]:
# def split_tags(tags_list):
#   tag_split_list = []
#   if(len(tags_list) != 0):
#     tags_lists = tags_list[0].replace(',',' ')
#     for tag in tags_lists.split(' '):
#         tag_split_list.append(tag)
#   return tag_split_list

# df['split_tags'] = df['tags'].apply(lambda x: split_tags(x))


In [144]:
# def final_split(x):
#   if(len(x.split_tags) != 0):
#     return (x.split_tags)
#   else:
#     return (x.description_split)

# df['final_split'] =  df.apply(lambda x:final_split(x), axis=1 )


In [225]:
from collections import Counter

vocab = Counter()
for items in df['description_split']:
       for word in items:
          vocab[word] += 1
vocab.most_common(20)

[('ویدیویی', 18),
 ('ورزشی', 17),
 ('ایران', 14),
 ('انتخاب', 14),
 ('پایگاه', 7),
 ('سیاسی', 7),
 ('پیکر', 7),
 ('ورزش', 6),
 ('موشک', 6),
 ('منبع', 6),
 ('وزیر', 5),
 ('روز', 5),
 ('شهرام', 5),
 ('عبدلی', 5),
 ('سازمان', 4),
 ('اسماعیل', 4),
 ('لحظه', 4),
 ('کروز', 4),
 ('پاوه', 4),
 ('سپاه', 4)]

In [226]:
def flatten(l):
    return [item for sublist in l for item in sublist]

# df['all_tags'] = df.apply (lambda row: flatten(row.tags) if len(row.tags)!=0 else row.lemmetize_description_remove_stopwords, axis=1)

processed_text = flatten([ i for i in df['description_split']])

vocabulary = list(set(processed_text))
vocabulary = [i for i in vocabulary if len(i)>1]


In [227]:
import numpy as np
import math
vocab_len = len(vocabulary)

weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

score = np.zeros((vocab_len),dtype=np.float32)
window_size = 3
covered_coocurrences = []

for i in range(0,vocab_len):
    score[i]=1
    for j in range(0,vocab_len):
        if j==i:
            weighted_edge[i][j]=0
        else:
            for window_start in range(0,(len(processed_text)-window_size+1)):
                
                window_end = window_start+window_size
                
                window = processed_text[window_start:window_end]
                
                if (vocabulary[i] in window) and (vocabulary[j] in window):
                    
                    index_of_i = window_start + window.index(vocabulary[i])
                    index_of_j = window_start + window.index(vocabulary[j]) 
                      
                    if [index_of_i,index_of_j] not in covered_coocurrences:
                        weighted_edge[i][j]+=1/math.fabs(index_of_i-index_of_j)
                        covered_coocurrences.append([index_of_i,index_of_j])


In [228]:
inout = np.zeros((vocab_len),dtype=np.float32)

for i in range(0,vocab_len):
    for j in range(0,vocab_len):
        inout[i]+=weighted_edge[i][j]

In [229]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(0,vocab_len):
        
        summation = 0
        for j in range(0,vocab_len):
            if weighted_edge[i][j] != 0:
                summation += (weighted_edge[i][j]/inout[j])*score[j]
                
        score[i] = (1-d) + d*(summation)
    
    if np.sum(np.fabs(prev_score-score)) <= threshold: #convergence condition
        print("Converging at iteration "+str(iter)+"....")
        break

Converging at iteration 32....


In [230]:
vocab_score_dict = {}
for i in range(0,vocab_len):
  vocab_score_dict[vocabulary[i]] = score[i]

dict(sorted(vocab_score_dict.items(), key=lambda item: item[1], reverse=True))

{'ایران': 5.7670646,
 'ویدیویی': 5.3162937,
 'ورزشی': 4.9785514,
 'انتخاب': 4.2360625,
 'پیکر': 3.7359998,
 'منبع': 3.654798,
 'موشک': 3.0792184,
 'ورزش': 2.8355887,
 'کنترل': 2.5847802,
 'پایگاه': 2.4812486,
 'روز': 2.4589555,
 'سیاسی': 2.454649,
 'همسر': 2.4223073,
 'شهرام': 2.3979995,
 'وقوع': 2.3698153,
 'مطهری': 2.3125057,
 'عبدلی': 2.3083684,
 'شهید': 2.2349508,
 'سازمان': 2.2250032,
 'وزیر': 2.2191322,
 'سپاه': 2.177409,
 'لحظه': 2.1746926,
 'دلیل': 2.1246254,
 'منطقه': 2.04301,
 'آمریکا': 2.006643,
 'قرار': 1.994342,
 'ساختمان': 1.9933096,
 'هدف': 1.9794285,
 'اسفند': 1.9504405,
 'پاوه': 1.9280425,
 'کروز': 1.8982887,
 'بیمارستان': 1.8708103,
 'جمعه': 1.8629674,
 'اسماعیل': 1.8621823,
 'زلزله': 1.8570931,
 'کشور': 1.8541588,
 'امروز': 1.849151,
 'خارجی': 1.8119409,
 'سرمایهگذاری': 1.7954824,
 'مراسم': 1.7901865,
 'زمین': 1.7854794,
 'برگزار': 1.7583336,
 'تهران': 1.7241628,
 'سقوط': 1.6942714,
 'بالگرد': 1.6551088,
 'حضور': 1.6228069,
 'دلاری': 1.6145722,
 'کرمان': 1.6122179,
 

In [231]:
phrases = []
phrase = " "

df['description_split_befor_remove_stopwprds'] = df['description'].apply(lambda x : x.split(' '))

processed_text2 = flatten([ i for i in df['description_split_befor_remove_stopwprds']])

for word in processed_text2:
    # if(word not in ['خبر','اخبار','فوری','سیاسی','فوری','سایت','جدیدترین','خبری','ورزشی','انتخاباتی','ویدیویی','تحلیلی','اخرین','خبرفوری','انتخابات','انتخاب']):
      if word in stopwords_plus:
          if phrase!= " ":
              phrases.append(str(phrase).strip().split())
          phrase = " "
      elif word not in stopwords_plus:
          phrase+=str(word)
          phrase+=" "

print(phrases)

[['مدیرعامل', 'سازمان', 'ورزش', 'شهرداری', 'تهران'], ['وضعیت', 'مجموعههای', 'ورزشی', 'تعطیل\u200cشده', 'شهرداری', 'تهران', 'توضیحاتی', 'ارائه'], ['درصد'], ['مجموعههای', 'ورزشی'], ['بسته'], ['مجددا', 'بازگشایی'], ['کوچکی'], ['مجموعههای', 'ورزشی'], ['بسته'], ['بازیکن', 'النصر', 'عربستان', 'ابرستاره', 'پرتغالی'], ['چالش'], ['مراسم', 'تشییع'], ['تدفین', 'اسماعیل', 'احمدی', 'مشاور', 'وزیر', 'ورزش'], ['امامزاده', 'اسماعیل', 'چیذر', 'برگزار'], ['فیلمی'], ['لحظه', 'شلیک', 'موشک', 'کروز'], ['پاوه', 'توسط', 'نیروی', 'هوافضای', 'سپاه'], ['مشاهده'], ['معماری', 'بناهای', 'ایران'], ['شهرت'], ['گرفتن', 'فال', 'حافظ'], ['تعبیر'], ['تفسیر'], ['علم'], ['سواد'], ['دلیل', 'تصمیم'], ['روز', 'تعبیر'], ['غزل\u200cهای', 'حافظ', 'شیرازی'], ['لحظه', 'انفجار', 'پمپ', 'هیدرولیک', 'بیل', 'مکانیکی', 'مقابل', 'صورت', 'راننده'], ['ویدیو'], ['ویدیویی'], ['نماز', 'خواندن', 'داماد', 'اصفهانی'], ['حاشیه', 'خیابان', 'کاوه'], ['منبع', 'تلخند', 'شوخی'], ['اسب'], ['اعصاب'], ['پرتاب'], ['مشاهده'], ['ویدیویی'], ['داستان', 'خوا

In [232]:
unique_phrases = []

for phrase in phrases:
    if phrase not in unique_phrases:
        unique_phrases.append(phrase)

print ("Unique Phrases (Candidate Keyphrases): \n")
print (unique_phrases)

Unique Phrases (Candidate Keyphrases): 

[['مدیرعامل', 'سازمان', 'ورزش', 'شهرداری', 'تهران'], ['وضعیت', 'مجموعههای', 'ورزشی', 'تعطیل\u200cشده', 'شهرداری', 'تهران', 'توضیحاتی', 'ارائه'], ['درصد'], ['مجموعههای', 'ورزشی'], ['بسته'], ['مجددا', 'بازگشایی'], ['کوچکی'], ['بازیکن', 'النصر', 'عربستان', 'ابرستاره', 'پرتغالی'], ['چالش'], ['مراسم', 'تشییع'], ['تدفین', 'اسماعیل', 'احمدی', 'مشاور', 'وزیر', 'ورزش'], ['امامزاده', 'اسماعیل', 'چیذر', 'برگزار'], ['فیلمی'], ['لحظه', 'شلیک', 'موشک', 'کروز'], ['پاوه', 'توسط', 'نیروی', 'هوافضای', 'سپاه'], ['مشاهده'], ['معماری', 'بناهای', 'ایران'], ['شهرت'], ['گرفتن', 'فال', 'حافظ'], ['تعبیر'], ['تفسیر'], ['علم'], ['سواد'], ['دلیل', 'تصمیم'], ['روز', 'تعبیر'], ['غزل\u200cهای', 'حافظ', 'شیرازی'], ['لحظه', 'انفجار', 'پمپ', 'هیدرولیک', 'بیل', 'مکانیکی', 'مقابل', 'صورت', 'راننده'], ['ویدیو'], ['ویدیویی'], ['نماز', 'خواندن', 'داماد', 'اصفهانی'], ['حاشیه', 'خیابان', 'کاوه'], ['منبع', 'تلخند', 'شوخی'], ['اسب'], ['اعصاب'], ['پرتاب'], ['داستان', 'خوانی', 'مرحوم', 'شهر

In [233]:
for word in vocabulary:
    #print word
    for phrase in unique_phrases:
        if (word in phrase) and ([word] in unique_phrases) and (len(phrase)>1):
            #if len(phrase)>1 then the current phrase is multi-worded.
            #if the word in vocabulary is present in unique_phrases as a single-word-phrase
            # and at the same time present as a word within a multi-worded phrase,
            # then I will remove the single-word-phrase from the list.
            unique_phrases.remove([word])
            
print ("Thinned Unique Phrases (Candidate Keyphrases): \n")
print (unique_phrases)  

Thinned Unique Phrases (Candidate Keyphrases): 

[['مدیرعامل', 'سازمان', 'ورزش', 'شهرداری', 'تهران'], ['وضعیت', 'مجموعههای', 'ورزشی', 'تعطیل\u200cشده', 'شهرداری', 'تهران', 'توضیحاتی', 'ارائه'], ['درصد'], ['مجموعههای', 'ورزشی'], ['بسته'], ['مجددا', 'بازگشایی'], ['کوچکی'], ['بازیکن', 'النصر', 'عربستان', 'ابرستاره', 'پرتغالی'], ['چالش'], ['مراسم', 'تشییع'], ['تدفین', 'اسماعیل', 'احمدی', 'مشاور', 'وزیر', 'ورزش'], ['امامزاده', 'اسماعیل', 'چیذر', 'برگزار'], ['فیلمی'], ['لحظه', 'شلیک', 'موشک', 'کروز'], ['پاوه', 'توسط', 'نیروی', 'هوافضای', 'سپاه'], ['مشاهده'], ['معماری', 'بناهای', 'ایران'], ['شهرت'], ['گرفتن', 'فال', 'حافظ'], ['تفسیر'], ['سواد'], ['دلیل', 'تصمیم'], ['روز', 'تعبیر'], ['غزل\u200cهای', 'حافظ', 'شیرازی'], ['لحظه', 'انفجار', 'پمپ', 'هیدرولیک', 'بیل', 'مکانیکی', 'مقابل', 'صورت', 'راننده'], ['ویدیو'], ['نماز', 'خواندن', 'داماد', 'اصفهانی'], ['حاشیه', 'خیابان', 'کاوه'], ['منبع', 'تلخند', 'شوخی'], ['اسب'], ['اعصاب'], ['داستان', 'خوانی', 'مرحوم', 'شهرام', 'عبدلی'], ['ماه'], ['دلیل', 'سق

In [ ]:
phrase_scores = []
keywords = []
for phrase in unique_phrases:
    phrase_score=0
    keyword = ''
    for word in phrase:
        if(len(word)>1):
          keyword += str(word)
          keyword += " "
          phrase_score+=score[vocabulary.index(word)]
    phrase_scores.append(phrase_score)
    keywords.append(keyword.strip())

i=0
phrase_dict = {}
for keyword in keywords:
    print ("Keyword: '"+str(keyword)+"', Score: "+str(phrase_scores[i]))
    phrase_dict[keyword] = phrase_scores[i]
    i+=1

In [249]:
import operator
sorted_dict = dict( sorted(phrase_dict.items(), key=operator.itemgetter(1),reverse=True))

In [262]:
sorted_dict

{'ویدیویی پیکر شهرام عبدلی بازیگر سینما': 16.362136483192444,
 'پیکر سرکار خانم اعظم عالیه روحانی همسر شهید آیتالله مرتضی مطهری': 15.805522859096527,
 'پیکر مرحوم اسماعیل احمدی مشاور وزیر ورزش': 14.744082927703857,
 'پیکر همسر شهید آیت\u200cالله مطهری عصر شنبه': 13.537166714668274,
 'وضعیت مجموعههای ورزشی تعطیل\u200cشده شهرداری تهران توضیحاتی ارائه': 12.329394161701202,
 'ویدیویی امانوئل مکرون رئیسجمهور فرانسه روز جمعه': 12.257254719734192,
 'موشک کروز زمینپایه ایران': 11.786328196525574,
 'خاطرات هاشمی رفسنجانی محسن سرپرست دفتر ریاست مجمع تشخیص مصلحت نظامآمد توضیحاتی': 11.624686419963837,
 'منبع ایرنا نیروهای امدادی ترکیه عملیات ریزش کنترل': 11.55662989616394,
 'ویدیویی سازمان سرمایهگذاری خارجی': 11.148720264434814,
 'سردار حاجی\u200cزاده فرمانده هوافضا سپاه پاسداران کشتن سربازان عینالاسد هدف': 10.853902339935303,
 'سیما لحظه شلیک موشک کروز پاوه': 10.79872316122055,
 'شناخت شرکای واقعی اقتصادی ایران منتشر': 10.267220497131348,
 'تدفین اسماعیل احمدی مشاور وزیر ورزش': 10.120447635650635

In [258]:
# load pickle module
import pickle

# create a binary pickle file 
f = open("file.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(sorted_dict,f)

# close file
f.close()

In [268]:
import io
with io.open('output.csv','w',encoding='utf-8-sig') as f:
    for key in sorted_dict.keys():
        f.write("%s,%s\n"%(key,sorted_dict[key]))